# Problem 2

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm,t
from scipy.optimize import minimize
import statsmodels as sm
import random
import scipy.optimize as optimize
from statsmodels.tsa import ar_model
import scipy

Problem1=pd.read_csv("problem1.csv")
Daily_Price = pd.read_csv("DailyPrices.csv")
portfolio= pd.read_csv("portfolio.csv")

In [21]:
# Problem 2
# Fit a Normal Distribution 
def norm_ES(x,alpha):
    miu = x.mean()
    x_adj=x-miu
    sigma = x_adj.std()
    norm_VaR = -norm.ppf(alpha,loc=miu,scale=sigma)
    
   
    ES_norm =- miu + sigma*norm.pdf(norm.ppf(alpha))/alpha
    print("When alpha = "+str(alpha)+" :")
    print("The VaR fitted a Normal Distribution is "+str(norm_VaR))
    print("The Expected Shortfall is "+str(ES_norm))
   
    return norm_VaR, ES_norm

# t distribution:
def MLE_t(pars, x):
    df = pars[0]
    loc=pars[1]
    scale = pars[2]
    ll = np.log(t.pdf(x, df=df,loc=loc,scale=scale)) 
    return -ll.sum()


def t_ES(x, alpha):
    mean_x=x.mean()
    std_x=x.std()
    cons = ({'type': 'ineq', 'fun': lambda x:  x[0] - 2}, {'type': 'ineq', 'fun': lambda x:  x[2]})
    model = optimize.minimize(fun = MLE_t,  x0 = [2, mean_x,std_x ], constraints=cons, args =x).x
    
    df=model[0]
    loc=model[1]
    scale=model[2]
    t_sample = t.rvs(df =df, loc = loc, scale = scale, size = 10000)
    
    t_VaR=-t.ppf(alpha, df =df, loc = loc, scale = scale)
    
    ES_t=-t_sample[t_sample<-t_VaR].mean()
    
    print("When alpha = "+str(alpha)+" :")
    print("The VaR fitted a T Distribution is "+str(t_VaR))
    print("The Expected Shortfall is "+str(ES_t))
    
    return  df,loc,scale,t_VaR,ES_t

def historic(data, N=100000, alpha=0.05):
    np.random.seed(50)
    simulated_draws = np.random.choice(data.iloc[:,0], size=N, replace=True)
    mu = simulated_draws.mean()
    
    sorted_data = np.sort(simulated_draws)
    
    VaR_index = int(len(sorted_data) * 0.05)
    VaR = -sorted_data[VaR_index]

    VaR_diff = VaR + mu
    
    ES = -sorted_data[sorted_data <= -VaR].mean()
    ES_diff = ES + mu
    
    
    return pd.DataFrame({"VaR Absolute": [VaR], 
                         "VaR Diff from Mean": [VaR_diff],
                         "ES Absolute": [ES], 
                         "ES Diff from Mean": [ES_diff]})


In [22]:
norm_VaR, ES_norm=norm_ES(Problem1['x'],0.05)
df,loc,scale,t_VaR,ES_t=t_ES(Problem1['x'],0.05)
historic(Problem1)


When alpha = 0.05 :
The VaR fitted a Normal Distribution is 0.08133532747633608
The Expected Shortfall is 0.10177426982904987
When alpha = 0.05 :
The VaR fitted a T Distribution is 0.07647580647535743
The Expected Shortfall is 0.11095120120352252


,VaR Absolute,VaR Diff from Mean,ES Absolute,ES Diff from Mean
0,0.075862,0.074978,0.115348,0.114465


# Problem 3

In [23]:
from risk_management.cal_return import return_calculate
from risk_management.copula import simulateCopula

In [24]:
prices = pd.read_csv('DailyPrices.csv')
returns = return_calculate(prices)
returns -= returns.mean(numeric_only=True)

returns = returns.copy()
returns = returns.drop('Date', axis=1)
returns.head()

,SPY,AAPL,MSFT,AMZN,TSLA,GOOGL,GOOG,META,NVDA,BRK-B,...,PNC,MDLZ,MO,ADI,GILD,LMT,SYK,GM,TFC,TJX
0,0.016251,0.023338,0.018706,0.010018,0.053757,0.009100,0.009403,0.015175,0.091347,0.006102,...,0.013489,-0.004152,0.004456,0.051421,0.001937,-0.013301,0.032537,0.026442,0.029295,0.012263
1,0.001245,-0.001203,-0.001004,0.011518,0.001507,0.009381,0.008869,-0.020165,0.000139,-0.001747,...,0.007438,-0.002499,0.005628,0.037956,0.007630,0.011219,0.002879,0.015504,-0.000665,-0.026958
2,-0.021237,-0.021083,-0.029119,-0.020450,-0.050476,-0.036633,-0.036584,-0.040761,-0.076056,-0.006660,...,-0.034268,0.005256,0.014881,-0.047911,-0.011518,0.003808,-0.031340,-0.031723,-0.032656,-0.029738
3,-0.006350,-0.009170,-0.009468,-0.011903,-0.021637,-0.015003,-0.012830,-0.007446,-0.035761,0.003979,...,0.000035,-0.000978,0.007067,-0.001360,-0.005579,-0.006967,-0.014157,-0.004304,-0.002953,0.014063
4,-0.010607,-0.017626,-0.000566,-0.014394,-0.040900,-0.003408,-0.007078,-0.019774,-0.011124,-0.002040,...,0.010176,0.007051,-0.009026,0.002320,-0.002810,-0.001698,0.007858,-0.037452,-0.001523,-0.014580


In [25]:
portfolio = pd.read_csv('portfolio.csv')
for stock in portfolio["Stock"]:
    portfolio.loc[portfolio['Stock'] == stock, 'Starting Price'] = prices.iloc[-1][stock]

portfolio.loc[portfolio['Portfolio'].isin(['A', 'B']), 'Distribution'] = 'T'
portfolio.loc[portfolio['Portfolio'] == 'C', 'Distribution'] = 'Normal'
print(portfolio.head())

  Portfolio Stock  Holding  Starting Price Distribution
0         A  AAPL       58      150.639999            T
1         A  TSLA       10      207.320007            T
2         A   JPM       64      140.419998            T
3         A    HD       27      315.549988            T
4         A   BAC      209       35.720001            T


In [26]:
portfolio_A = portfolio[portfolio['Portfolio'] == 'A'].copy()
portfolio_A = portfolio_A.drop('Portfolio', axis=1)
portfolio_B = portfolio[portfolio['Portfolio'] == 'B'].copy()
portfolio_B = portfolio_B.drop('Portfolio', axis=1)
portfolio_C = portfolio[portfolio['Portfolio'] == 'C'].copy()
portfolio_C = portfolio_C.drop('Portfolio', axis=1)

stocks_in_A = portfolio_A['Stock'].tolist()
returns_A = returns[stocks_in_A]
stocks_in_B = portfolio_B['Stock'].tolist()
returns_B = returns[stocks_in_B]
stocks_in_C = portfolio_C['Stock'].tolist()
returns_C = returns[stocks_in_C]

print(portfolio_A.shape)
print(returns_A.shape)

(35, 4)
(248, 35)


In [27]:
risk_A = simulateCopula(portfolio_A, returns_A)
print('Portfolio A:')
print(risk_A.iloc[-1] if isinstance(risk_A, pd.DataFrame) else risk_A[-1])

risk_B = simulateCopula(portfolio_B, returns_B)
print('Portfolio B:')
print(risk_B.iloc[-1] if isinstance(risk_B, pd.DataFrame) else risk_B[-1])

risk_C = simulateCopula(portfolio_C, returns_C)
print('Portfolio C:')
print(risk_C.iloc[-1] if isinstance(risk_C, pd.DataFrame) else risk_C[-1])

Portfolio A:
Stock               Total
VaR95         8005.002384
ES95         10634.445194
VaR95_Pct        0.026688
ES95_Pct         0.035454
Name: 35, dtype: object
Portfolio B:
Stock              Total
VaR95        6873.195227
ES95         9071.337269
VaR95_Pct       0.023348
ES95_Pct        0.030814
Name: 32, dtype: object
Portfolio C:
Stock              Total
VaR95        5912.689935
ES95         7425.745754
VaR95_Pct       0.021895
ES95_Pct        0.027498
Name: 32, dtype: object
